In [2]:
# First attempt at logistic regression, step by step.

import pandas as pd
import numpy as np
import os
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('../../../../GitHub_Data/Dota2Matches/match_outcomes.csv')
df.head()

,match_id,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,start_time,parser_version,win,rad
0,1636204962,34549,0,0,-51743434,-120875154,1437014585,12,1,0
1,1636204962,0,61598,138825,0,207232,1437014585,12,0,1
2,1636322679,0,-44943233,-240360907,19599,0,1437019968,12,0,0
3,1636322679,-97530201,0,0,0,-116349387,1437019968,12,1,1
4,1637385965,0,0,0,104738,0,1437052551,12,1,0


In [7]:
# Messy. Let's just work with player info. drop parser_version and start_time, for starters
df.drop(['parser_version', 'start_time'], axis=1, inplace=True)

In [8]:
df.columns

Index(['match_id', 'account_id_0', 'account_id_1', 'account_id_2',
       'account_id_3', 'account_id_4', 'win', 'rad'],
      dtype='object')

In [9]:
df.shape

(1828588, 8)

In [13]:
df = df.melt(id_vars=['match_id','win','rad'], \
    value_vars=['account_id_0', 'account_id_1', 'account_id_2', 'account_id_3', 'account_id_4'], \
        var_name='PlayerNumber', value_name='PlayerAccountID')

df.head()

,match_id,win,rad,PlayerNumber,PlayerAccountID
0,1636204962,1,0,account_id_0,34549
1,1636204962,0,1,account_id_0,0
2,1636322679,0,0,account_id_0,0
3,1636322679,1,1,account_id_0,-97530201
4,1637385965,1,0,account_id_0,0


In [14]:
dfPlayerRatings = pd.read_csv('../../../../GitHub_Data/Dota2Matches/player_ratings.csv')
dfPlayerRatings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,236579,14,24,27.868035,5.212361
1,-343,1,1,26.544163,8.065475
2,-1217,1,1,26.521103,8.114989
3,-1227,1,1,27.248025,8.092217
4,-1284,0,1,22.931016,8.092224


In [15]:
df = pd.merge(df,dfPlayerRatings,how='left', left_on='PlayerAccountID', right_on='account_id')
df.head()

,match_id,win,rad,PlayerNumber,PlayerAccountID,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,1636204962,1,0,account_id_0,34549,34549,10,17,25.496820,5.277160
1,1636204962,0,1,account_id_0,0,0,1608398,3315071,25.000000,8.333333
2,1636322679,0,0,account_id_0,0,0,1608398,3315071,25.000000,8.333333
3,1636322679,1,1,account_id_0,-97530201,-97530201,1,1,26.880745,8.118755
4,1637385965,1,0,account_id_0,0,0,1608398,3315071,25.000000,8.333333


In [16]:
# so the blank slot doesn't really mean anything. All those 0s are just empty spots where somebody dropped.
# We'll get rid of them.

df.drop(['account_id'], axis=1, inplace=True)
df = df[df['PlayerAccountID'] != 0]
df.head()

,match_id,win,rad,PlayerNumber,PlayerAccountID,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,1636204962,1,0,account_id_0,34549,10,17,25.496820,5.277160
3,1636322679,1,1,account_id_0,-97530201,1,1,26.880745,8.118755
6,1637623870,1,0,account_id_0,-123447796,22,47,24.867150,3.543024
7,1637623870,0,1,account_id_0,-108454938,28,46,32.028790,3.659563
8,1637739731,0,0,account_id_0,320093,9,15,26.680175,5.869468


In [17]:
# The question becomes - how to create a 'team' from the individuals?
# For now, let's just add the trueskill_mu for each player and ignore the sigma
# why add and not average? mostly because this is just a test and i'm curious of the results

df.drop(['trueskill_sigma', 'PlayerNumber', 'PlayerAccountID', 'total_wins', 'total_matches'], axis=1, inplace=True)
df = df.groupby(['match_id', 'win', 'rad']).sum()
df.head()

trueskill_mu
match_id   win rad              
1636204962 0   1       74.975331
           1   0       76.073306
1636322679 0   0       76.288377
           1   1       58.561092
1637385965 0   1       47.955511

In [ ]:
# Brilliant, now we just need a way to separate out the teams.
# We need match_id, rad_win, rad_trueskill, dire_trueskill.
# then we predict rad_win using rad_trueskill, dire_trueskill
# match_id could be helpful for tying in additional datasets later.

# Project for tomorrow, it's 10pm and i'm tired, need to go to work tomorrow.